In [26]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import mini_data.images as image
from tqdm import tqdm

((train_set, train_label), (test_set, test_label)) = image.load_data()
train_set, train_label = train_set[:20000], train_label[:20000]

def calcSqDistances(X,Kmus):
  return((-2 * X @ Kmus.T + np.sum(Kmus * Kmus, axis = 1).T).T + np.sum(X * X, axis = 1)).T

def determineRnk(sqDmat):
  low = np.argmin(sqDmat, axis = 1)
  rnk = np.identity(sqDmat.shape[1])[low]
  return rnk

def recalcMus(X, Rnk):
    Kmus = np.zeros((len(Rnk[0]), 2))
    return (np.divide(X.T.dot(Rnk), np.sum(Rnk, axis=0))).T

def runKMeans(K):
    X = train_set
    N = np.shape(X)[0]
    D = np.shape(X)[1]

    Kmus = np.zeros((K,D))

    rand_inds = np.random.permutation(N)
    Kmus = X[rand_inds[0:K],:]

    for itr in range(1000):
        distmat = calcSqDistances(X, Kmus)

        rnk = determineRnk(distmat)

        mus_old = Kmus
        Kmus = recalcMus(X, rnk)
        if np.sum(np.abs(mus_old.reshape((-1,1)) - Kmus.reshape((-1,1)))) < 1e-6:
            break

    return Kmus

def get_cluster_plot(Kmus, rows = 2): 
    '''
    : get a plot of what different clusters represent
    : takes as input the cluster location matrix
    '''
    fig,axs = plt.subplots(rows,5, figsize = (15,9))
    for i in range(len(Kmus)):
        cluster = Kmus[i].reshape(28,-1) * 255
        axs[i//5][i%5].imshow(cluster, cmap = plt.cm.binary)
        axs[i//5][i%5].set_title(f'Cluster {i + 1}')


ModuleNotFoundError: No module named 'cv2'

In [ ]:
test = runKMeans(100)

IndexError: tuple index out of range